In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
import os
import librosa
from librosa import display,util,load,feature

In [3]:
!pip install librosa

  Using cached librosa-0.8.0.tar.gz (183 kB)
  Using cached resampy-0.2.2.tar.gz (323 kB)
  Using cached SoundFile-0.10.3.post1-py2.py3.cp26.cp27.cp32.cp33.cp34.cp35.cp36.pp27.pp32.pp33-none-win_amd64.whl (689 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201381 sha256=9c00992be9a2d63441542addb70fdca235c366be270a533eba0e8bf65a13fd0b
  Stored in directory: c:\users\rujut\appdata\local\pip\cache\wheels\32\2c\ce\86e49d4769aceba728421c24c0d726054bf4ca01175ff42bdd
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23146 sha256=aca603e9e4ece7f2336b8bf2396007e585c33af8fa41cf28f5678426c0ea2c86
  Stored in directory: c:\users\rujut\appdata\local\pip\cache\wheels\de\14\0a\863e4ed680b3204444cf486733e609d7ff7abe8fceafab67dc
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320724 sha256=3f466cc578f320be35f4815820f408338256e10242344c670b426482337c3851
  Store

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import make_pipeline,Pipeline

In [3]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [4]:
from keras import layers, optimizers
from keras.models import model_from_json
from keras.models import Sequential, Model
from keras.optimizers import Adam, Adamax # sgd
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import Dense, LSTM, Dropout, Flatten, Input, Conv1D, MaxPooling1D
from keras.activations import relu, elu, softmax, selu
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
os.getcwd()

'C:\\Users\\rujut'

In [7]:
os.chdir((r'C:\Users\rujut\Desktop\Sleigh Group'))

In [8]:
df = pd.read_csv("sample_combine2.csv")

In [9]:
df.id.unique().shape

(1211,)

In [10]:
df.head()

,id,mfcc,mfcc.1,mfcc.2,mfcc.3,mfcc.4,mfcc.5,mfcc.6,mfcc.7,mfcc.8,...,contrast.3,contrast.4,contrast.5,contrast.6,tonnetz,tonnetz.1,tonnetz.2,tonnetz.3,tonnetz.4,tonnetz.5
0,10118,-273.600494,103.989548,-33.254021,46.472401,-36.957596,7.668534,-26.684517,-21.350235,-2.519242,...,18.374612,20.101481,21.955693,62.391661,0.012897,0.034230,-0.120919,0.076604,-0.035556,-0.015480
1,10118,-272.701630,105.984398,-47.450043,44.580639,-39.831207,13.744248,-30.243664,-15.632530,-7.305216,...,16.629601,18.645249,19.424802,63.568005,-0.040606,-0.009914,0.073158,0.017044,0.000726,0.023044
2,10118,-254.748016,98.882637,-27.369404,42.244934,-33.880486,9.336581,-33.618820,-14.017242,-8.014104,...,16.945670,18.159030,18.835621,63.558387,0.008319,0.023324,0.046921,0.034156,-0.002578,0.000598
3,10118,-281.164032,93.107315,-35.382385,61.903549,-27.928452,15.431595,-30.133783,-18.698492,-2.843096,...,17.165046,19.571505,20.503783,63.006274,0.016291,-0.050295,0.093024,0.153118,0.037379,-0.024679
4,10118,-280.229126,85.746666,-37.427578,57.026058,-43.526890,19.675930,-35.488728,-10.556077,-10.283919,...,16.542366,18.740579,19.698682,66.071237,0.010619,-0.014958,0.017122,0.070661,0.009914,-0.004792


In [11]:
x = df.drop('id', axis=1)
y = df['id']

In [12]:
print(x)

              mfcc      mfcc.1     mfcc.2     mfcc.3     mfcc.4     mfcc.5  \
0      -273.600494  103.989548 -33.254021  46.472401 -36.957596   7.668534   
1      -272.701630  105.984398 -47.450043  44.580639 -39.831207  13.744248   
2      -254.748016   98.882637 -27.369404  42.244934 -33.880486   9.336581   
3      -281.164032   93.107315 -35.382385  61.903549 -27.928452  15.431595   
4      -280.229126   85.746666 -37.427578  57.026058 -43.526890  19.675930   
...            ...         ...        ...        ...        ...        ...   
148548 -276.699524  136.624802 -30.104067  56.355434  -9.107129  12.288788   
148549 -371.456604  133.546921 -89.245056  28.143431 -40.675251  -2.348424   
148550 -376.359222  127.453705 -90.675835  31.803614 -44.774052  -4.925930   
148551 -329.837494  144.583969 -22.814005  53.807255 -14.585546  18.426785   
148552 -336.411621  150.438873 -19.194506  40.251331 -31.157505  18.909906   

           mfcc.6     mfcc.7     mfcc.8     mfcc.9  ...  contra

In [13]:
print(y)

0         10118
1         10118
2         10118
3         10118
4         10118
          ...  
148548    10814
148549    10814
148550    10814
148551    10814
148552    10814
Name: id, Length: 148553, dtype: int64


In [14]:
# Train, test, split into training and testing
x_train1, x_test, y_train1, y_test = train_test_split(x, y, test_size = 0.1, random_state = 121)

In [15]:
# Split the training data into training and validation
x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, test_size = 0.2, random_state = 121)

In [16]:
x_val.shape

(26740, 193)

In [17]:
x_test.shape

(14856, 193)

In [18]:
x_train.shape

(106957, 193)

In [19]:
x_train.head()

,mfcc,mfcc.1,mfcc.2,mfcc.3,mfcc.4,mfcc.5,mfcc.6,mfcc.7,mfcc.8,mfcc.9,...,contrast.3,contrast.4,contrast.5,contrast.6,tonnetz,tonnetz.1,tonnetz.2,tonnetz.3,tonnetz.4,tonnetz.5
122479,-111.835106,123.529839,-57.270351,54.344002,-35.500778,-0.112050,-34.001778,3.176709,-16.178476,3.627999,...,18.596113,18.749638,21.911079,68.882176,-0.046061,-0.012110,0.014788,0.055449,-0.016177,0.007080
89696,-279.739044,159.028107,-6.478727,27.223921,5.381705,-6.302465,-8.855764,1.581423,-5.878119,4.527397,...,17.312557,17.659842,19.857674,55.333804,-0.025748,0.035453,0.137811,0.070776,-0.014212,-0.034011
6315,-311.175598,129.620682,-30.142984,46.297081,-5.227465,14.518489,-33.710239,-8.294282,-12.262257,-12.805036,...,15.913652,18.771085,17.124063,61.198795,0.027925,-0.013749,-0.048303,-0.004185,-0.000168,-0.005338
115595,-284.976471,130.115219,-69.154236,49.471706,-12.652008,14.319530,-31.729902,-9.170115,-10.469703,0.124704,...,17.402675,19.949399,21.637098,63.563526,0.041466,0.020593,-0.021872,0.021393,0.005392,0.008102
112745,-458.117493,138.954895,-6.648188,40.384029,-2.157960,22.178341,-8.865500,5.281638,-12.471731,-4.515971,...,15.986117,17.234774,18.021560,63.373429,0.012017,0.035412,-0.028299,0.022332,0.014092,0.004994


In [20]:
#reshape x datasets from 2D into 3D arrays
x_train_array = x_train.values # convert df to array
x_train_ = np.reshape(x_train_array, newshape = (106957, 1,193))
x_train_.shape # (106957, 1, 193)

(106957, 1, 193)

In [21]:
x_val_array = x_val.values # convert df to array
x_val_ = np.reshape(x_val_array, newshape = (26740, 1,193))
x_val_.shape # (26740, 1,193)

(26740, 1, 193)

In [22]:
x_test_array = x_test.values # convert df to array
x_test_ = np.reshape(x_test_array, newshape = (14856, 1, 193))
x_test_.shape # (14856, 1, 193)

(14856, 1, 193)

In [23]:
# reshape y data for last layer in models (softmax activation function)
y_train_cat = pd.get_dummies(y_train) # converts variables in one column to have their own column
y_train_ = y_train_cat.values 
y_train_.shape # (106957, 1211)

(106957, 1211)

In [24]:
y_train_cat.columns

Int64Index([10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009,
            10010,
            ...
            11242, 11243, 11244, 11245, 11246, 11247, 11248, 11249, 11250,
            11251],
           dtype='int64', length=1211)

In [25]:
y_val_cat = pd.get_dummies(y_val) # converts variables in one column to have their own column
y_val_ = y_val_cat.values 
y_val_.shape  # (26740, 1211)

(26740, 1211)

In [26]:
y_test_cat = pd.get_dummies(y_test) # converts variables in one column to have their own column
y_test_ = y_test_cat.values 
y_test_.shape # (14856, 1208)

(14856, 1208)

In [27]:
# Set seed to get same results
np.random.seed(121)
# Define model as Sequential class
model_cnn_selu = Sequential()

In [28]:
# Block 1

# first layer: convolutional layer with filter size of 2 and strid of 2, due to 
# the features having shape divisable by 2. selu shown to work better than relu
# and leaky relu without CNN.
# Max Pool to get only the important features
# add dropout to combat overfitting. rate = 10% of neurons to drop

model_cnn_selu.add(Conv1D(128, (2), strides=(2), padding='same', input_shape = (1,193), activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))
model_cnn_selu.add(Dropout(rate = 0.1))

# Block 2
model_cnn_selu.add(Conv1D(256, (2), strides=(2), padding='same', activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))
model_cnn_selu.add(Dropout(rate = 0.1))

# Block 3
model_cnn_selu.add(Conv1D(256, (2), strides=(2), padding='same', activation= 'selu'))
model_cnn_selu.add(Conv1D(512, (2), strides=(2), padding='same',activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))

# Block 4
model_cnn_selu.add(Conv1D(512, (2), strides=(2), padding='same', activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))
model_cnn_selu.add(Dropout(rate = 0.1))

# Block 5
model_cnn_selu.add(Conv1D(512, (2), strides=(2), padding='same',activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))

# Block 7
model_cnn_selu.add(Conv1D(256, (2), strides=(2), padding='same', activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))

# Block 8 
model_cnn_selu.add(Conv1D(256, (2), strides=(2), padding='same', activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))
model_cnn_selu.add(Dropout(rate = 0.1))

# Block 9
model_cnn_selu.add(Conv1D(256, (2), strides=(2), padding='same', activation= 'selu'))
model_cnn_selu.add(MaxPooling1D((2), strides=(2), padding='same'))

# Output Block
model_cnn_selu.add(layers.Flatten())
model_cnn_selu.add(Dense(1211, activation = 'softmax'))

In [29]:
# Compile the model with a specified Adamax optimizer
opt = Adamax(lr = 1e-3, decay = 1e-5) # Adamax has shown to yield faster learning than Adam and SGD
model_cnn_selu.compile(loss = 'categorical_crossentropy', 
              optimizer = opt,
             metrics = ['accuracy'])

In [30]:

# Add automated stopping after val_loss difference from epoch t and t-1 is 
# more than 0.001; give it three more epochs to try and get back on track (patience)
earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              patience=3,
                              verbose=0, mode='auto')

In [38]:
# fit the data and save it with history variable. Accuracy 0.7565
history = model_cnn_selu.fit(x_train_, 
                    y_train_, 
                    epochs = 30, 
                    batch_size = 300,
                    validation_data= (x_val_,y_val_), callbacks = [earlystop])

Epoch 1/30
357/357 [==============================] - 3113s 9s/step - loss: 6.1167 - accuracy: 0.0461 - val_loss: 5.0902 - val_accuracy: 0.1179
Epoch 2/30
357/357 [==============================] - 184s 514ms/step - loss: 4.7170 - accuracy: 0.1501 - val_loss: 4.2290 - val_accuracy: 0.2125
Epoch 3/30
357/357 [==============================] - 182s 511ms/step - loss: 4.0805 - accuracy: 0.2230 - val_loss: 3.7323 - val_accuracy: 0.2809
Epoch 4/30
357/357 [==============================] - 192s 538ms/step - loss: 3.6430 - accuracy: 0.2827 - val_loss: 3.3542 - val_accuracy: 0.3429
Epoch 5/30
357/357 [==============================] - 215s 602ms/step - loss: 3.2942 - accuracy: 0.3356 - val_loss: 3.0738 - val_accuracy: 0.3908
Epoch 6/30
357/357 [==============================] - 216s 605ms/step - loss: 3.0078 - accuracy: 0.3827 - val_loss: 2.8673 - val_accuracy: 0.4257
Epoch 7/30
357/357 [==============================] - 190s 533ms/step - loss: 2.7763 - accuracy: 0.4187 - val_loss: 2.6555 - v

In [33]:
#Best Model to be saved
history = model_cnn_selu.fit(x_train_, 
                    y_train_, 
                    epochs = 100, 
                    batch_size = 100,
                    validation_data= (x_val_,y_val_), callbacks = [earlystop])

Epoch 1/100
1070/1070 [==============================] - 191s 179ms/step - loss: 0.9340 - accuracy: 0.7526 - val_loss: 1.6015 - val_accuracy: 0.6936
Epoch 2/100
1070/1070 [==============================] - 198s 185ms/step - loss: 0.8871 - accuracy: 0.7644 - val_loss: 1.6058 - val_accuracy: 0.6965
Epoch 3/100
1070/1070 [==============================] - 221s 206ms/step - loss: 0.8633 - accuracy: 0.7699 - val_loss: 1.5822 - val_accuracy: 0.7020
Epoch 4/100
1070/1070 [==============================] - 206s 193ms/step - loss: 0.8375 - accuracy: 0.7746 - val_loss: 1.5890 - val_accuracy: 0.7020
Epoch 5/100
1070/1070 [==============================] - 209s 195ms/step - loss: 0.8119 - accuracy: 0.7809 - val_loss: 1.5826 - val_accuracy: 0.7064
Epoch 6/100
1070/1070 [==============================] - 217s 202ms/step - loss: 0.7814 - accuracy: 0.7865 - val_loss: 1.5781 - val_accuracy: 0.7101
Epoch 7/100
1070/1070 [==============================] - 216s 202ms/step - loss: 0.7615 - accuracy: 0.7932

In [40]:
history = model_cnn_selu.fit(x_train_, 
                    y_train_, 
                    epochs = 60, 
                    batch_size = 200,
                    validation_data= (x_val_,y_val_), callbacks = [earlystop])

Epoch 1/60
535/535 [==============================] - 178s 332ms/step - loss: 0.5865 - accuracy: 0.8368 - val_loss: 1.5021 - val_accuracy: 0.7288
Epoch 2/60
535/535 [==============================] - 184s 344ms/step - loss: 0.6024 - accuracy: 0.8322 - val_loss: 1.4948 - val_accuracy: 0.7240
Epoch 3/60
535/535 [==============================] - 188s 351ms/step - loss: 0.5924 - accuracy: 0.8331 - val_loss: 1.5200 - val_accuracy: 0.7247
Epoch 4/60
535/535 [==============================] - 188s 352ms/step - loss: 0.5785 - accuracy: 0.8383 - val_loss: 1.4961 - val_accuracy: 0.7311
Epoch 5/60
535/535 [==============================] - 192s 358ms/step - loss: 0.5647 - accuracy: 0.8389 - val_loss: 1.5042 - val_accuracy: 0.7313


In [32]:
history = model_cnn_selu.fit(x_train_, 
                    y_train_, 
                    epochs = 100, 
                    batch_size = 200,
                    validation_data= (x_val_,y_val_), callbacks = [earlystop])

Epoch 1/100
535/535 [==============================] - 206s 385ms/step - loss: 2.4298 - accuracy: 0.4764 - val_loss: 2.4272 - val_accuracy: 0.5050
Epoch 2/100
535/535 [==============================] - 219s 409ms/step - loss: 2.2570 - accuracy: 0.5062 - val_loss: 2.2672 - val_accuracy: 0.5393
Epoch 3/100
535/535 [==============================] - 224s 418ms/step - loss: 2.1180 - accuracy: 0.5320 - val_loss: 2.1919 - val_accuracy: 0.5547
Epoch 4/100
535/535 [==============================] - 206s 385ms/step - loss: 1.9993 - accuracy: 0.5537 - val_loss: 2.0947 - val_accuracy: 0.5731
Epoch 5/100
535/535 [==============================] - 200s 373ms/step - loss: 1.8824 - accuracy: 0.5750 - val_loss: 2.0136 - val_accuracy: 0.5869
Epoch 6/100
535/535 [==============================] - 200s 373ms/step - loss: 1.7884 - accuracy: 0.5914 - val_loss: 1.9632 - val_accuracy: 0.5994
Epoch 7/100
535/535 [==============================] - 198s 371ms/step - loss: 1.7062 - accuracy: 0.6069 - val_loss: 1

In [46]:
print(tf.__version__)

2.3.1


In [45]:
import tensorflow as tf

In [47]:
import keras as k
print(k.__version__)

2.4.3


In [34]:
# Save best model so don't have to rerun in scenario of crash
model_json = model_cnn_selu.to_json()
with open("model_1201.json", "w") as json_file:
    json_file.write(model_json)
    
model_cnn_selu.save_weights("model_weights_1201.h5")
print("Saved model to drive")

Saved model to drive


In [35]:
# Use model to predict on testing set
y_pred = model_cnn_selu.predict(x_test_)

In [36]:
y_pred[0]

array([2.6121163e-09, 4.3498733e-07, 9.1678658e-07, ..., 2.8244358e-09,
       4.4911015e-12, 3.8307621e-11], dtype=float32)

In [37]:
# Convert y_pred from continuous variables to discreet binary (0,1)
yy = []
for elem in y_pred: # each element is an array
    row = []
    for e in elem: # each number in the array
        if e == elem.max(): # since softmax outputs probability...
            row.append(1) # assign highest prob as 1
        else:
            row.append(0) # assign all other probs as 0
    yy.append(row)

In [38]:
y_pred_ = np.array(yy)

In [39]:
len(y_pred_[0])

1211

In [40]:
len(y_test_[0])

1208